In [3]:
# =========================================
# Lending Club Risk Predictor Prototype
# =========================================
# Author: Louis Ko
# Description:
#   - 一鍵啟動 FastAPI + ngrok + 測試 API
#   - 自動清除舊 ngrok、建立新通道、驗證回應
# =========================================
!pip install python-dotenv
import os, time, platform, requests
from pyngrok import ngrok
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()
print("NGROK_AUTHTOKEN =", os.getenv("NGROK_AUTHTOKEN"))
# 從 .env 取得 ngrok authtoken
NGROK_TOKEN = os.getenv("NGROK_AUTHTOKEN")

if not NGROK_TOKEN:
    raise EnvironmentError("缺少環境變數 NGROK_AUTHTOKEN，請在 .env 設定中加入。設定完後請重新啟動")

# =========================================
# STEP 1 安裝套件
# =========================================
!pip install fastapi uvicorn pyngrok tensorflow scikit-learn pandas numpy seaborn matplotlib joblib langchain openai --quiet

# =========================================
# STEP 2 終止舊 ngrok 通道
# =========================================
print("清除舊 ngrok 通道...")
if platform.system() == "Windows":
    os.system("taskkill /f /im ngrok.exe 2>nul")
else:
    os.system("pkill -f ngrok || true")

try:
    ngrok.kill()
except Exception:
    pass
print("舊 ngrok 通道已清除。")

# =========================================
# STEP 3 啟動 FastAPI
# =========================================
print("啟動 FastAPI 服務中...")
os.system("uvicorn api.main:app --host 0.0.0.0 --port 8000 &")

# =========================================
# STEP 4 登入 ngrok 並建立公開通道
# =========================================
ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(8000)
print("Public API URL:", public_url.public_url)

# 等待伺服器啟動
for i in range(15, 0, -3):
    print(f"等待伺服器啟動中...（{i} 秒）")
    time.sleep(3)

# =========================================
# STEP 5 測試 /predict API
# =========================================
sample = {
    "loan_amnt": 25000,
    "annual_inc": 100000,
    "dti": 25.5,
    "total_acc": 15,
    "revol_util": 42.3,
    "int_rate": 10.5
}

url = public_url.public_url + "/predict"

print("\n測試 /predict API ...")
try:
    r = requests.post(url, json=sample, timeout=10)
    print("Status:", r.status_code)
    print("Response:", r.json())
except Exception as e:
    print("API 測試失敗:", e)

# =========================================
# STEP 6 LangChain Tool 測試
# =========================================
from langchain.tools import tool

@tool
def risk_predict_tool(customer_info: dict):
    """呼叫貸款風險預測 API"""
    import requests
    response = requests.post(url, json=customer_info)
    return response.json()

print("\nAgent Tool 測試結果：")
print(risk_predict_tool.invoke({"customer_info": sample}))




[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
NGROK_AUTHTOKEN = 3wesnq6XqsKJU1ZHzZ88o_7HrePPESrpda5suSBeKTy

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
清除舊 ngrok 通道...
舊 ngrok 通道已清除。
啟動 FastAPI 服務中...
Public API URL: https://d2a7830a5bbf.ngrok-free.app
等待伺服器啟動中...（15 秒）


2025-11-09 01:04:01.176259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


等待伺服器啟動中...（12 秒）
等待伺服器啟動中...（9 秒）
使用模型目錄：/Users/louisko/fileproject/Wits-Hackathon/ml_module/model
載入模型與前處理器中...
模型與前處理器載入完成。


INFO:     Started server process [72487]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


等待伺服器啟動中...（6 秒）
等待伺服器啟動中...（3 秒）

測試 /predict API ...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
INFO:     2401:e180:8840:ab2b:959c:a0b:7b15:c200:0 - "POST /predict HTTP/1.1" 200 OK
Status: 200
Response: {'risk_score': 0.7023876309394836, 'risk_level': 'Medium', 'model_columns': ['loan_amnt', 'annual_inc', 'dti', 'total_acc', 'revol_util', 'int_rate']}

Agent Tool 測試結果：
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
INFO:     2401:e180:8840:ab2b:959c:a0b:7b15:c200:0 - "POST /predict HTTP/1.1" 200 OK
{'risk_score': 0.7023876309394836, 'risk_level': 'Medium', 'model_columns': ['loan_amnt', 'annual_inc', 'dti', 'total_acc', 'revol_util', 'int_rate']}
